<a id=“title_ID”></a>
# JWST Pipeline Validation Testing Notebook: Spec2, flat_field, MIRI
Instruments Affected: MIRI

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#JWST-Pipeline-Validation-Testing-Notebook:-Spec2,-flat_field,-MIRI" data-toc-modified-id="JWST-Pipeline-Validation-Testing-Notebook:-Spec2,-flat_field,-MIRI-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>JWST Pipeline Validation Testing Notebook: Spec2, flat_field, MIRI</a></span><ul class="toc-item"><li><span><a href="#Introduction-and-summary-of-test-being-run" data-toc-modified-id="Introduction-and-summary-of-test-being-run-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction and summary of test being run</a></span></li><li><span><a href="#Documentation" data-toc-modified-id="Documentation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Documentation</a></span></li><li><span><a href="#Data-used" data-toc-modified-id="Data-used-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Data used</a></span><ul class="toc-item"><li><span><a href="#Create-temporary-directory" data-toc-modified-id="Create-temporary-directory-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Create temporary directory</a></span></li><li><span><a href="#Set-up-import-statements" data-toc-modified-id="Set-up-import-statements-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Set up import statements</a></span></li><li><span><a href="#Print-pipeline-version-number" data-toc-modified-id="Print-pipeline-version-number-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Print pipeline version number</a></span></li><li><span><a href="#Read-in-data-from-artifactory" data-toc-modified-id="Read-in-data-from-artifactory-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Read in data from artifactory</a></span></li><li><span><a href="#Read-in-input-image-as-JWST-data-model" data-toc-modified-id="Read-in-input-image-as-JWST-data-model-1.3.5"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>Read in input image as JWST data model</a></span></li><li><span><a href="#Run-output-of-calwebb_detector1-through-calwebb_spec2" data-toc-modified-id="Run-output-of-calwebb_detector1-through-calwebb_spec2-1.3.6"><span class="toc-item-num">1.3.6&nbsp;&nbsp;</span>Run output of calwebb_detector1 through calwebb_spec2</a></span></li><li><span><a href="#Create-image-models-of-rate-and-cal-files" data-toc-modified-id="Create-image-models-of-rate-and-cal-files-1.3.7"><span class="toc-item-num">1.3.7&nbsp;&nbsp;</span>Create image models of rate and cal files</a></span></li><li><span><a href="#Get-flat_field-reference-file" data-toc-modified-id="Get-flat_field-reference-file-1.3.8"><span class="toc-item-num">1.3.8&nbsp;&nbsp;</span>Get flat_field reference file</a></span></li><li><span><a href="#Create-image-lists-for-looping" data-toc-modified-id="Create-image-lists-for-looping-1.3.9"><span class="toc-item-num">1.3.9&nbsp;&nbsp;</span>Create image lists for looping</a></span></li><li><span><a href="#Check-DQ-flagging" data-toc-modified-id="Check-DQ-flagging-1.3.10"><span class="toc-item-num">1.3.10&nbsp;&nbsp;</span>Check DQ flagging</a></span></li><li><span><a href="#Calculate-the-rate/cal-image-ratio" data-toc-modified-id="Calculate-the-rate/cal-image-ratio-1.3.11"><span class="toc-item-num">1.3.11&nbsp;&nbsp;</span>Calculate the rate/cal image ratio</a></span></li><li><span><a href="#Compare-flat-field-reference-file-with-the-rate/cal-image-ratio-and-check-that-they-are-equal" data-toc-modified-id="Compare-flat-field-reference-file-with-the-rate/cal-image-ratio-and-check-that-they-are-equal-1.3.12"><span class="toc-item-num">1.3.12&nbsp;&nbsp;</span>Compare flat field reference file with the rate/cal image ratio and check that they are equal</a></span></li></ul></li><li><span><a href="#About-this-Notebook" data-toc-modified-id="About-this-Notebook-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>About this Notebook</a></span></li></ul></li></ul></div>

## Introduction and summary of test being run

This notebook processes an image through calwebb_spec2 and examines the output of the flat_field step. The steps are as follow:

1) Set up data path and directory and image file names.

2) Run output of calwebb_detector1 through the flat_field step in calwebb_spec2.

3) Get flat field reference file from CRDS. 

4) Compare the flat field reference file with the rate/cal image ratio and check that they are the same.

## Documentation

The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

## Data used 

The data used in this test is a simulated MIRI image created using MIRISim. The documentation for MIRISim can be found here: https://wiki.miricle.org/bin/view/Public/MIRISim_Public?

Author: Mike Engesser (Adapted from Tea Temim's calwebb-image1 notebook)

### Create temporary directory

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)
print(data_dir)

<TemporaryDirectory '/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpzk7ujyoy'>


In [2]:
import os
if 'CRDS_CACHE_TYPE' in os.environ:
    if os.environ['CRDS_CACHE_TYPE'] == 'local':
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif os.path.isdir(os.environ['CRDS_CACHE_TYPE']):
        os.environ['CRDS_PATH'] = os.environ['CRDS_CACHE_TYPE']
print('CRDS cache location: {}'.format(os.environ['CRDS_PATH']))

CRDS cache location: /grp/crds/cache


### Set up import statements

In [3]:
from astropy.io import fits, ascii
import pytest
import numpy as np
import numpy.ma as ma

import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline
from jwst.datamodels import RampModel, ImageModel, dqflags
from jwst.pipeline import calwebb_image2
from jwst.pipeline import calwebb_spec2

from ci_watson.artifactory_helpers import get_bigdata
import crds
import os

In [4]:
# Specify CRDS locations and pmap
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['TEST_BIGDATA']='https://bytesalad.stsci.edu/artifactory/'

### Print pipeline version number

In [5]:
jwst.__version__ 

'1.5.3'

### Read in data from artifactory

In [6]:
file1 = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFUSHORT_12LONGexp1_rate.fits')
file2 = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFUSHORT_12MEDIUMexp1_rate.fits')
file3 = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFUSHORT_12SHORTexp1_rate.fits')
file4 = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFULONG_34LONGexp1_rate.fits')
file5 = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFULONG_34MEDIUMexp1_rate.fits')
file6 = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFULONG_34SHORTexp1_rate.fits')


### Read in input image as JWST data model

In [7]:
im1 = ImageModel(file1)
im2 = ImageModel(file2)
im3 = ImageModel(file3)
im4 = ImageModel(file4)
im5 = ImageModel(file5)
im6 = ImageModel(file6)

### Run output of calwebb_detector1 through calwebb_spec2

In [8]:
def run_spec2(im):
    """Creates and runs an instance of the JWST spec2 pipeline. """
    
    spec2 = calwebb_spec2.Spec2Pipeline()
    spec2.bkg_subtract.skip = True
    spec2.assign_wcs.skip = False
    spec2.imprint_subtract.skip = True
    spec2.msa_flagging.skip = True
    spec2.extract_2d.skip = True
    spec2.wavecorr.skip = True
    spec2.srctype.skip = True
    spec2.straylight.skip = True
    spec2.fringe.skip = True
    spec2.pathloss.skip = True
    spec2.barshadow.skip = True
    spec2.photom.skip = True
    spec2.resample_spec.skip = True
    spec2.cube_build.skip = True
    spec2.extract_1d.skip = True
    spec2.flat_field.skip = False
    spec2.save_results = True
    
    spec2.run(im)

In [9]:
run_spec2(im1)
run_spec2(im2)
run_spec2(im3)
run_spec2(im4)
run_spec2(im5)
run_spec2(im6)

2022-07-06 15:19:35,387 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.


2022-07-06 15:19:35,389 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2022-07-06 15:19:35,391 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2022-07-06 15:19:35,393 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.


2022-07-06 15:19:35,394 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.


...

2022-07-06 15:20:30,614 - stpipe.Spec2Pipeline - INFO - Finished processing product det_image_seq1_MIRIFULONG_34SHORTexp1_rate


2022-07-06 15:20:30,614 - stpipe.Spec2Pipeline - INFO - Ending calwebb_spec2


2022-07-06 15:20:30,615 - stpipe.Spec2Pipeline - INFO - Results used CRDS context: jwst_0914.pmap


2022-07-06 15:20:31,365 - stpipe.Spec2Pipeline - INFO - Saved model in det_image_seq1_MIRIFULONG_34SHORTexp1_cal.fits


2022-07-06 15:20:31,366 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline done


### Create image models of rate and cal files

In [10]:
# Get names of cal files
input_file1 = file1.replace('rate.fits', 'cal.fits')
input_file2 = file2.replace('rate.fits', 'cal.fits')
input_file3 = file3.replace('rate.fits', 'cal.fits')
input_file4 = file4.replace('rate.fits', 'cal.fits')
input_file5 = file5.replace('rate.fits', 'cal.fits')
input_file6 = file6.replace('rate.fits', 'cal.fits')

In [11]:
# create image models of rate and cal files
im_cal1 = ImageModel(input_file1)
im_cal2 = ImageModel(input_file2)
im_cal3 = ImageModel(input_file3)
im_cal4 = ImageModel(input_file4)
im_cal5 = ImageModel(input_file5)
im_cal6 = ImageModel(input_file6)

im_rate1 = ImageModel(file1)
im_rate2 = ImageModel(file2)
im_rate3 = ImageModel(file3)
im_rate4 = ImageModel(file4)
im_rate5 = ImageModel(file5)
im_rate6 = ImageModel(file6)

### Get flat_field reference file

In [12]:
flatreffile1 = im_cal1.meta.ref_file.flat.name
flatreffile2 = im_cal2.meta.ref_file.flat.name
flatreffile3 = im_cal3.meta.ref_file.flat.name
flatreffile4 = im_cal4.meta.ref_file.flat.name
flatreffile5 = im_cal5.meta.ref_file.flat.name
flatreffile6 = im_cal6.meta.ref_file.flat.name

print('Flat reference file', flatreffile1)
print('Flat reference file', flatreffile2)
print('Flat reference file', flatreffile3)
print('Flat reference file', flatreffile4)
print('Flat reference file', flatreffile5)
print('Flat reference file', flatreffile6)
# find location of file
basename1 = crds.core.config.pop_crds_uri(flatreffile1)
basename2 = crds.core.config.pop_crds_uri(flatreffile2)
basename3 = crds.core.config.pop_crds_uri(flatreffile3)
basename4 = crds.core.config.pop_crds_uri(flatreffile4)
basename5 = crds.core.config.pop_crds_uri(flatreffile5)
basename6 = crds.core.config.pop_crds_uri(flatreffile6)

path1 = crds.locate_file(basename1, "jwst")
path2 = crds.locate_file(basename2, "jwst")
path3 = crds.locate_file(basename3, "jwst")
path4 = crds.locate_file(basename4, "jwst")
path5 = crds.locate_file(basename5, "jwst")
path6 = crds.locate_file(basename6, "jwst")
# open reference file
flat_im1 = ImageModel(path1)
flat_im2 = ImageModel(path2)
flat_im3 = ImageModel(path3)
flat_im4 = ImageModel(path4)
flat_im5 = ImageModel(path5)
flat_im6 = ImageModel(path6)

Flat reference file crds://jwst_miri_flat_0753.fits
Flat reference file crds://jwst_miri_flat_0763.fits
Flat reference file crds://jwst_miri_flat_0749.fits
Flat reference file crds://jwst_miri_flat_0750.fits
Flat reference file crds://jwst_miri_flat_0747.fits
Flat reference file crds://jwst_miri_flat_0758.fits


### Create image lists for looping

In [13]:
file_list = [file1, file2, file3, file4, file5, file6]
im_list = [im_cal1, im_cal2, im_cal3, im_cal4, im_cal5, im_cal6]
rate_list = [im_rate1, im_rate2, im_rate3, im_rate4, im_rate5, im_rate6]
flat_list = [flat_im1, flat_im2, flat_im3, flat_im4, flat_im5, flat_im6]

### Check DQ flagging 
Any pixel flagged as NON_SCIENCE should also be flagged as DO_NOT_USE. Check if this is in place in both the input reference file and for the output science file of the calwebb_image2 pipeline. If there are no assert errors, the test below passes.

In [14]:
# Check if the output cal file is flagged properly
# return DQ arrays for masking
def check_dq_flags(im_cal, flat_im):
    # Test that all pixels flagged with NON_SCIENCE are also flagged as DO_NOT_USE
    nonsciencearray = (im_cal.dq & dqflags.pixel['NON_SCIENCE'] > 0)
    badarray = (im_cal.dq & dqflags.pixel['DO_NOT_USE'] > 0)
    try:
        assert nonsciencearray.all() == badarray.all()
    except:
        print("AssertionError: NON_SCIENCE array not equal to DO_NOT_USE array in science data.")

    # Test if the input reference file had the flags all set the same way
    nonsciencearray = (flat_im.dq & dqflags.pixel['NON_SCIENCE'] > 0)
    badarray = (flat_im.dq & dqflags.pixel['DO_NOT_USE'] > 0)
    try:
        assert nonsciencearray.all() == badarray.all()
    except:
        print("AssertionError: NON_SCIENCE array not equal to DO_NOT_USE array in reference data.")
    
    return badarray

badarrays = [check_dq_flags(im, flat) for im,flat in zip(im_list, flat_list)]

### Calculate the rate/cal image ratio

In [15]:
def get_ratios(im, rate, mask):
    # compute and return the ratio of the rate and cal images
    rate_mask = ma.masked_array(rate.data, mask=mask)
    ratio = im.data/rate_mask
    
    return ratio

ratios = [get_ratios(cal_im, rate_im, badarray) for cal_im, rate_im, badarray in zip(im_list, rate_list, badarrays)]

2022-07-06 15:20:40,102 - stpipe - WARNING - /internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.10/site-packages/asdf/tags/core/ndarray.py:559: RuntimeWarning: invalid value encountered in divide
  return getattr(self._make_array(), name)(*args)



### Compare flat field reference file with the rate/cal image ratio and check that they are equal

The minimum and maximum values should be 1.0, implying all values are 1, and the images are equal.

In [16]:
def check_flat(ratio_im, flat_im):
    return ratio_im/flat_im.data

check_flats = [check_flat(ratio, flat) for ratio, flat in zip(ratios, flat_list)]

In [17]:
def check_min_max(check_flat_im, im_name):
    print('Minimum and maximum values for {} image are: {} and {}'.format(im_name, np.nanmin(check_flat_im), np.nanmax(check_flat_im)))
    #assert np.nanmin(check_flat_im) == 1.0 and np.nanmax(check_flat_im) == 1.0

for im, name in zip(check_flats, file_list):
    im_name = name.split('/')[-1]
    check_min_max(im, im_name)

Minimum and maximum values for det_image_seq1_MIRIFUSHORT_12LONGexp1_rate.fits image are: 1.0 and 1.0
Minimum and maximum values for det_image_seq1_MIRIFUSHORT_12MEDIUMexp1_rate.fits image are: 1.0 and 1.0
Minimum and maximum values for det_image_seq1_MIRIFUSHORT_12SHORTexp1_rate.fits image are: 1.0 and 1.0
Minimum and maximum values for det_image_seq1_MIRIFULONG_34LONGexp1_rate.fits image are: 1.0 and 1.0
Minimum and maximum values for det_image_seq1_MIRIFULONG_34MEDIUMexp1_rate.fits image are: 1.0 and 1.0
Minimum and maximum values for det_image_seq1_MIRIFULONG_34SHORTexp1_rate.fits image are: 1.0 and 1.0


<a id="about_ID"></a>
## About this Notebook
**Author:** Mike Engesser, Science Support Analyst I, INS
<br>**Updated On:** 06/22/2021